In [5]:
import pandas as pd
from statsbombpy import sb
import numpy as np
from mplsoccer import Sbopen

In [6]:
parser = Sbopen()
df_competitions = parser.competition()

for competition_id, season_id in zip(df_competitions['competition_id'], df_competitions['season_id']):
    print(f"Processing Competition ID: {competition_id}, Season ID: {season_id}")


Processing Competition ID: 9, Season ID: 281
Processing Competition ID: 9, Season ID: 27
Processing Competition ID: 1267, Season ID: 107
Processing Competition ID: 16, Season ID: 4
Processing Competition ID: 16, Season ID: 1
Processing Competition ID: 16, Season ID: 2
Processing Competition ID: 16, Season ID: 27
Processing Competition ID: 16, Season ID: 26
Processing Competition ID: 16, Season ID: 25
Processing Competition ID: 16, Season ID: 24
Processing Competition ID: 16, Season ID: 23
Processing Competition ID: 16, Season ID: 22
Processing Competition ID: 16, Season ID: 21
Processing Competition ID: 16, Season ID: 41
Processing Competition ID: 16, Season ID: 39
Processing Competition ID: 16, Season ID: 37
Processing Competition ID: 16, Season ID: 44
Processing Competition ID: 16, Season ID: 76
Processing Competition ID: 16, Season ID: 277
Processing Competition ID: 16, Season ID: 71
Processing Competition ID: 16, Season ID: 276
Processing Competition ID: 223, Season ID: 282
Process

In [4]:
df_matches = {}
df_competitions_filtered = df_competitions[df_competitions['competition_id'] == 9]
for competition_id, season_id in zip(df_competitions_filtered['competition_id'], df_competitions['season_id']):
    df_match = parser.match(competition_id=competition_id, season_id=season_id)
    
    for i,match_id in enumerate(df_match['match_id']):
        df_matches[match_id] = {}
        try:
            df_matches[match_id]['event']=sb.events(match_id=match_id) 
            print(competition_id,season_id,match_id)
        except Exception as e:
            print(f"Failed to fetch data for Match ID {match_id}: {e}")


9 281 3895302
9 281 3895292
9 281 3895333
9 281 3895340
9 281 3895348
9 281 3895286
9 281 3895220
9 281 3895250
9 281 3895266
9 281 3895275
9 281 3895180
9 281 3895134
9 281 3895121
9 281 3895107
9 281 3895074
9 281 3895139
9 281 3895086
9 281 3895320
9 281 3895309
9 281 3895258
9 281 3895244
9 281 3895232
9 281 3895210
9 281 3895202
9 281 3895194
9 281 3895182
9 281 3895167
9 281 3895158
9 281 3895153
9 281 3895113
9 281 3895095
9 281 3895067
9 281 3895060
9 281 3895052
9 27 3890561
9 27 3890505
9 27 3890511
9 27 3890515
9 27 3890411
9 27 3890397
9 27 3890401
9 27 3890396
9 27 3890384
9 27 3890385
9 27 3890282
9 27 3890287
9 27 3890269
9 27 3890268
9 27 3890402
9 27 3890564
9 27 3890563
9 27 3890562
9 27 3890560
9 27 3890559
9 27 3890558
9 27 3890557
9 27 3890556
9 27 3890555
9 27 3890554
9 27 3890553
9 27 3890552
9 27 3890551
9 27 3890550
9 27 3890549
9 27 3890548
9 27 3890547
9 27 3890546
9 27 3890545
9 27 3890544
9 27 3890543
9 27 3890542
9 27 3890541
9 27 3890540
9 27 3890539
9 27

In [4]:
df_matches = pd.DataFrame(df_matches)
df_matches.to_csv('matches.csv', index=False)

In [7]:
def is_point_inside_triangle(point, v1, v2, v3):
    x, y = point
    x1, y1 = v1
    x2, y2 = v2
    x3, y3 = v3

    denominator = ((y2 - y3) * (x1 - x3) + (x3 - x2) * (y1 - y3))
    a = ((y2 - y3) * (x - x3) + (x3 - x2) * (y - y3)) / denominator
    b = ((y3 - y1) * (x - x3) + (x1 - x3) * (y - y3)) / denominator
    c = 1 - a - b

    return 0 <= a <= 1 and 0 <= b <= 1 and 0 <= c <= 1

In [8]:
def extract_goalkeeper_and_players(freeze_frame, shot_location):
    try:
        goalkeeper_location = None
        num_opposing_players = 0
        goal_left_post = [120, 36] 
        goal_right_post = [120, 44]
                    
        if isinstance(freeze_frame, list):
            for player in freeze_frame:
                location = player['location']
                is_opponent = not player['teammate']
                            
                if is_opponent and player['position']['name'] == 'Goalkeeper':
                    goalkeeper_location = location
                            
                if is_opponent and is_point_inside_triangle(location, shot_location, goal_left_post, goal_right_post):
                    num_opposing_players += 1
                    
        return goalkeeper_location, num_opposing_players
    except Exception:
        return np.nan,np.nan

In [9]:
df_shot = pd.DataFrame(columns=[
    'competition_id', 'season_id', 'match_id','location', 'shot_outcome', 
    'shot_statsbomb_xg', 'under_pressure', 'shot_type', 'shot_body_part', 'shot_one_on_one', 
    'shot_freeze_frame'
])

for competition_id, season_id in zip(df_competitions['competition_id'], df_competitions['season_id']):
    df_match = sb.matches(competition_id=competition_id, season_id=season_id)
    
    for match_id in df_match['match_id']:
        try:
            event_data = sb.events(match_id=match_id)
            print(competition_id,season_id,match_id)
            mask_shot = (event_data['type'] == 'Shot')
            
            columns_to_extract = [
                'location', 'shot_outcome', 
                'shot_statsbomb_xg', 'under_pressure', 'shot_type', 'shot_body_part', 'shot_one_on_one', 
                'shot_freeze_frame'
            ]
            
            existing_columns = [col for col in columns_to_extract if col in event_data.columns]
            
            shots_temp = event_data.loc[mask_shot, existing_columns]
            
            shots_temp['competition_id'] = competition_id
            shots_temp['season_id'] = season_id
            shots_temp['match_id'] = match_id

            shots_temp['x'] = shots_temp['location'].apply(lambda loc: loc[0] if isinstance(loc, list) and len(loc) > 0 else None)
            shots_temp['y'] = shots_temp['location'].apply(lambda loc: loc[1] if isinstance(loc, list) and len(loc) > 1 else None)
            
            results = shots_temp.apply(
                lambda row: extract_goalkeeper_and_players(row['shot_freeze_frame'], [row['x'], row['y']]) if 'shot_freeze_frame' in existing_columns and row['shot_freeze_frame'] is not np.nan else (np.nan, np.nan), axis=1
            )
            shots_temp['goalkeeper_location'], shots_temp['num_opposing_players'] = zip(*results)

            shots_temp['x_gk'] = shots_temp['goalkeeper_location'].apply(lambda loc: loc[0] if isinstance(loc, list) and len(loc) > 0 else np.nan)
            shots_temp['y_gk'] = shots_temp['goalkeeper_location'].apply(lambda loc: loc[1] if isinstance(loc, list) and len(loc) > 1 else np.nan)
            
            df_shot = pd.concat([df_shot, shots_temp], ignore_index=True)
        
        except KeyError as e:
            print(f"Missing data for Match ID {match_id}: {e}")
        except Exception as e:
            print(f"Error processing Match ID {match_id}: {e}")

9 281 3895302


C:\Users\ADAM\AppData\Local\Temp\ipykernel_7612\682825995.py:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_shot = pd.concat([df_shot, shots_temp], ignore_index=True)


9 281 3895292
9 281 3895333
9 281 3895340
9 281 3895348
9 281 3895286
9 281 3895220
9 281 3895250
9 281 3895266
9 281 3895275
9 281 3895180
9 281 3895134
9 281 3895121
9 281 3895107
9 281 3895074
9 281 3895139
9 281 3895086
9 281 3895320
9 281 3895309
9 281 3895258
9 281 3895244
9 281 3895232
9 281 3895210
9 281 3895202
9 281 3895194
9 281 3895182
9 281 3895167
9 281 3895158
9 281 3895153
9 281 3895113
9 281 3895095
9 281 3895067
9 281 3895060
9 281 3895052
9 27 3890561
9 27 3890505
9 27 3890511
9 27 3890515
9 27 3890411
9 27 3890397
9 27 3890401
9 27 3890396
9 27 3890384
9 27 3890385
9 27 3890282
9 27 3890287
9 27 3890269
9 27 3890268
9 27 3890402
9 27 3890564
9 27 3890563
9 27 3890562
9 27 3890560
9 27 3890559
9 27 3890558
9 27 3890557
9 27 3890556
9 27 3890555
9 27 3890554
9 27 3890553
9 27 3890552
9 27 3890551
9 27 3890550
9 27 3890549
9 27 3890548
9 27 3890547
9 27 3890546
9 27 3890545
9 27 3890544
9 27 3890543
9 27 3890542
9 27 3890541
9 27 3890540
9 27 3890539
9 27 3890538
9 27 

In [10]:
df_shot.head()

,competition_id,season_id,match_id,location,shot_outcome,shot_statsbomb_xg,under_pressure,shot_type,shot_body_part,shot_one_on_one,shot_freeze_frame,x,y,goalkeeper_location,num_opposing_players,x_gk,y_gk
0,9,281,3895302,"[100.4, 35.1]",Blocked,0.056644,NaN,Open Play,Right Foot,NaN,"[{'location': [101.9, 35.1], 'player': {'id': ...",100.4,35.1,"[118.4, 38.5]",2.0,118.4,38.5
1,9,281,3895302,"[114.6, 33.5]",Saved,0.143381,True,Open Play,Left Foot,NaN,"[{'location': [92.1, 63.8], 'player': {'id': 8...",114.6,33.5,"[118.3, 36.4]",1.0,118.3,36.4
2,9,281,3895302,"[106.2, 55.8]",Blocked,0.038188,NaN,Open Play,Left Foot,NaN,"[{'location': [118.2, 42.9], 'player': {'id': ...",106.2,55.8,"[118.2, 42.9]",2.0,118.2,42.9
3,9,281,3895302,"[113.9, 47.4]",Blocked,0.052781,NaN,Open Play,Head,NaN,"[{'location': [117.1, 37.7], 'player': {'id': ...",113.9,47.4,"[118.9, 41.9]",1.0,118.9,41.9
4,9,281,3895302,"[89.2, 42.5]",Blocked,0.021272,True,Open Play,Left Foot,NaN,"[{'location': [103.8, 39.7], 'player': {'id': ...",89.2,42.5,"[116.9, 40.1]",3.0,116.9,40.1


In [11]:
df_shot.drop(columns='shot_freeze_frame', inplace=True)

In [ ]:
df_shot['goal'] = df_shot.apply(lambda row:1 if row['shot_outcome']=='Goal' else 0, axis=1)

In [12]:
df_shot.to_csv('shots_data.csv', index=False)